# Measurement of Loudspeaker to Sequential Spherical Microphone Array

This notebook sketches how to sequentially measure impulse responses of spherical microphone arrays (SMAs) or equatorial microphone arrays (EMAs) with the VariSphear device.

* Two sweeps are played, one for the analog feedback and one to the
loudspeaker.
* Two channels are recorded, the analog feedback and the VariSphear microphone.
* After each measurement, the VariSphear rotates to the next microphone
position on the specified microphone grid.

## Load requirements

In [ ]:
# Automatically reload the acoustics_hardware package if anything is changed
%load_ext autoreload
%autoreload 2
# fmt: off

import sys
from datetime import datetime, timedelta
from pathlib import Path
from time import sleep

import numpy as np
import sounddevice as sd
from IPython.display import Audio, display, Markdown
from ipywidgets import Output
from pyfar.samplings import sph_gaussian, sph_lebedev, sph_fliege
from scipy.io import savemat
from scipy.signal.waveforms import chirp

from acoustics_hardware import utils
from acoustics_hardware.processors import deconvolve
from acoustics_hardware.serial import VariSphere

# Prepare output widgets
widget_preview_str = '<h3><center><font color="green">{}</font></center></h3>'
widget_preview_rec = Output()
widget_preview_rec.layout.border = "1px solid green"
widget_preview_ir = Output()
widget_preview_ir.layout.border = "3px solid green"

## Set up audio hardware

Adjust your settings for the individual audio hardware!

In [ ]:
# Reinitialize in case sounddevice was reconnected
# noinspection PyProtectedMember
sd._terminate()
# noinspection PyProtectedMember
sd._initialize()

# Show list of available audio devices
print(sd.query_devices())

# fmt: off
# Audio device configuration
device_name = "Orion 32"               # e.g. on macOS
# device_name = "Orion32 ASIO Driver"  # e.g. on Windows
fs = 48000                 # in Hz
device_output_desc = "Type / name of the loudspeaker and amplifier"
device_output_ch = [1, 2]  # output channel mapping (IDs starting from 1)
device_input_desc = "DPA 4060 in VariSphear scattering body; Antelope MP32"
device_input_ch = [1, 2]   # input channel mapping (IDs starting from 1)
# The audio device name and sampling frequency as well as input and output channels
# should be adjusted according to the utilized measurement hardware. The first channel
# in the input and output mapping thereby specify the loopback channel (the recorded
# reference signal which will be used for the deconvolution of the other channels).
# NOTE: It is currently not supported to not use a loopback channel, e.g. in case the
#       measurement hardware does not allow it. This should be implemented in the future!

# Sweep configuration
meas_room = "Name / description of the room"
meas_comment = "Further description of the environment / configuration"
meas_iterations = 2               # measurement iterations per position that will be averaged
sweep_hp, sweep_lp = 50, 20e3     # in Hz
sweep_amplitude = -40             # in dB_FS
sweep_win_in_periods = 1          # in periods at the start frequency
sweep_win_out_periods = 6         # in periods at the end frequency
sweep_duration = [0.2, 2.0, 0.8]  # in s, duration of [pre-silence, sweep, post-silence]
# The frequency range and amplitude should be adjusted according to the utilized sound
# source. The number of fade-in and fade-out oscillation periods can be adjusted in
# order to not yield clicks at the start and end of the sweep (verify with the
# spectrogram of the resulting loopback measurement). The sweep duration should be
# adjusted according to the present environmental / room conditions (a longer sweep
# will yield a better signal-to-noise ratio in the deconvolved impulse responses). The
# specified post-silence time will also impose a limit on the length of the captured
# microphone signals and should therefore allow the reverberation to fully decay in all
# frequency bands.

# Deconvolution configuration
deconv_type = "lin"              # "lin" or "cyc"
deconv_hp, deconv_lp = 10, 21e3  # in Hz
deconv_inv_dyn = None            # in dB
deconv_ir_len_s = 1.0            # in s
deconv_win_in_len = 64           # in samples
deconv_win_out_len = 64          # in samples
deconv_ir_amp = 0                # in dB
# The deconvolution type should be chosen based on keeping ("cyc") or cutting out ("lin")
# non-harmonic distortion products from the resulting impulse responses. Either the
# frequency range or the inversion dynamic range should be restricted in order to yield
# sensible impulse responses. The length of the deconvolved impulse responses should be
# adjusted based on the room conditions to preserve the reverberation in all frequency
# bands. The resulting deconvolved impulse responses may be amplified.

# fmt: on
print(f'\nCheck audio device "{device_name}" ... ', end="")
try:
    sd.check_output_settings(
        device=device_name,
        channels=len(device_output_ch),  # as number
        samplerate=fs,                   # in Hz
    )
    sd.check_input_settings(
        device=device_name,
        channels=len(device_input_ch),   # as number
        samplerate=fs,                   # in Hz
    )
except (ValueError, sd.PortAudioError) as e:
    sys.exit(e)
print("done.")

# Compute IR length from given time in seconds
deconv_ir_len = int(deconv_ir_len_s * fs)

print(f"\nGenerate sweep ... ", end="")
data_out = chirp(
    t=np.arange(np.ceil(sweep_duration[1] * fs)) / fs,  # in s
    f0=sweep_hp,                                        # in Hz
    t1=sweep_duration[1],                               # in s
    f1=sweep_lp,                                        # in Hz
    method="logarithmic",
    phi=90,  # in deg (to start with a sample at 0)
)                                                       # as [samples,]
# Apply amplitude scaling
data_out *= 10 ** (sweep_amplitude / 20)  # as factor
# Apply fade-in and fade-out window
data_out = utils.fade_signal(
    sig=data_out,
    win_in_len=int(sweep_win_in_periods * fs / sweep_hp),
    win_out_len=int(sweep_win_out_periods * fs / sweep_lp),
)
# Prepend and append zeroes
data_out = np.pad(
    array=data_out,
    pad_width=(int(sweep_duration[0] * fs), int(sweep_duration[2] * fs)),
)
data_out_is_adjusted = False
print("done.")

## Test audio hardware

Initialize the audio hardware and estimate the inherent system latency.

In [ ]:
%matplotlib inline

if data_out_is_adjusted:
    # noinspection PyUnresolvedReferences,PyUnboundLocalVariable
    display(Markdown(widget_preview_str.format(
        f"Using detected latency of {data_rec_latency} samples")))
else:
    # Determine latency if it has not been done yet
    print(f"\nRecord sweep ... ", end="")
    data_rec = sd.playrec(
        data=data_out,                    # as [samples,]
        samplerate=fs,                    # in Hz
        device=device_name,
        input_mapping=device_input_ch,    # channel numbers start from 1
        output_mapping=device_output_ch,  # channel numbers start from 1
        blocking=True,
    ).T                                   # as [channels, samples]
    print("done.")

    # Determine latency by initial zeroes in the recording
    # (this value will be used in the measurement section later)
    data_rec_latency = int(
        np.min(np.argmax(np.abs(data_rec) > 0, axis=-1))
    )  # in samples
    display(Markdown(widget_preview_str.format(
        f"Adjust for detected latency of {data_rec_latency} samples")))

    # Append zeroes to excitation signal based on determined latency
    data_out = np.pad(data_out, pad_width=(0, data_rec_latency))
    data_out_is_adjusted = True
    sleep(1)  # in s

print(f"\nRecord sweep ... ", end="")
data_rec = sd.playrec(
    data=data_out,                    # as [samples,]
    samplerate=fs,                    # in Hz
    device=device_name,
    input_mapping=device_input_ch,    # channel numbers start from 1
    output_mapping=device_output_ch,  # channel numbers start from 1
    blocking=True,
).T                                   # as [channels, samples]
print("done.")

# Cut initial zeroes based on determined latency
data_rec = data_rec[:, data_rec_latency:]  # in samples

# Print recorded peak and individual RMS levels
print(utils.generate_levels_string(data_rec))

display(Markdown("<h2><center>Excitation signal</center></h2>"))
utils.plot_data(data=data_out, fs=fs, ch_labels=device_input_ch[0])

display(Markdown("<h2><center>Recorded signals</center></h2>"))
utils.plot_data(data=data_rec, fs=fs, ch_labels=device_input_ch)
for ch in range(data_rec.shape[0]):
    display(
        Markdown(f"<h3>Channel in_{device_input_ch[ch]}</h3>"),
        Audio(data_rec[ch, :], rate=fs),
    )

data_ir = deconvolve(
    exc_sig=data_rec[:1],            # first channel as loopback
    rec_sig=data_rec[1:],
    fs=fs,                           # in Hz
    f_low=deconv_hp,                 # in Hz
    f_high=deconv_lp,                # in Hz
    res_len=deconv_ir_len / fs,      # in s
    deconv_type=deconv_type,
    inv_dyn_db=deconv_inv_dyn,       # in dB
    win_in_len=deconv_win_in_len,    # in samples
    win_out_len=deconv_win_out_len,  # in samples
)                                    # as [channels, samples]
if not data_ir.shape[0]:
    sys.exit("No deconvolved data available (only loopback channel recorded).")

display(Markdown("<h2><center>Deconvolved Impulse Responses</center></h2>"))
utils.plot_data(data=data_ir, fs=fs, ch_labels=device_input_ch[1:])
if data_ir.ndim > 1 and data_ir.shape[0] > 1:
    utils.plot_data(data=data_ir, fs=fs,
                    ch_labels=device_input_ch[1:], is_stacked=True)

## Setup and test turntable

Initialize the VariSphear and move the turntable to an arbitrary position.

In [ ]:
# VariSphere configuration
vari_ip = "192.168.127.120"  # default value
# For a 1:1 Ethernet connection, set your local IPv4 to an adjacent address
# e.g. "192.168.127.1" and your subnet mask to "255.255.255.0"

print(f"\nConnect to VariSphear at {vari_ip} ... ", end="")
vari = VariSphere(
    az_port="4001",  # default value
    el_port="4002",  # default value
    ip=vari_ip,
)
try:
    vari.az.check_pc_mc_communication()
    vari.el.check_pc_mc_communication()
except Exception as e:
    sys.exit(e)
print("done.")

# Try different angles (in deg) here
az, el = 0, 0  # in deg
print(f"\nMove VariSphear to {az=:.1f}°, {el=:.1f}° ... ", end="")
vari.move_blocking(az=az, el=el)
print("done.")

## Prepare measurement grid

Adjust your settings to the desired grid!

In [ ]:
%matplotlib widget

# fmt: off
# SMA grid configuration
array_r = 0.085                                  # in m
# array_grid_type, array_sh_order = "SMA_GL", 2  # generate Gauss-Legendre grid, 18 positions
# array_grid_type, array_sh_order = "SMA_GL", 4  # generate Gauss-Legendre grid, 50 positions
# array_grid_type, array_sh_order = "SMA_GL", 8  # generate Gauss-Legendre grid, 162 positions
# array_grid_type, array_sh_order = "SMA_GL", 12 # generate Gauss-Legendre grid, 338 positions
# array_grid_type, array_sh_order = "SMA_GL", 29 # generate Gauss-Legendre grid, 1800 positions
# array_grid_type, array_sh_order = "SMA_GL", 44 # generate Gauss-Legendre grid, 4050 positions
# array_grid_type, array_sh_order = "SMA_LE", 2  # generate Lebedev grid, 14 positions
# array_grid_type, array_sh_order = "SMA_LE", 4  # generate Lebedev grid, 38 positions
# array_grid_type, array_sh_order = "SMA_LE", 8  # generate Lebedev grid, 110 positions
# array_grid_type, array_sh_order = "SMA_LE", 12 # generate Lebedev grid, 230 positions
# array_grid_type, array_sh_order = "SMA_LE", 29 # generate Lebedev grid, 1202 positions
array_grid_type, array_sh_order = "SMA_LE", 44  # generate Lebedev grid, 2702 positions
# array_grid_type, array_sh_order = "SMA_FM", 2  # generate Fliege-Maier grid, 9 positions
# array_grid_type, array_sh_order = "SMA_FM", 4  # generate Fliege-Maier grid, 25 positions
# array_grid_type, array_sh_order = "SMA_FM", 8  # generate Fliege-Maier grid, 81 positions
# array_grid_type, array_sh_order = "SMA_FM", 12 # generate Fliege-Maier grid, 169 positions
# array_grid_type, array_sh_order = "SMA_FM", 29 # generate Fliege-Maier grid, 900 positions
# array_grid_type, array_sh_order = "EMA", 2     # generate equatorial grid, 5 positions
# array_grid_type, array_sh_order = "EMA", 4     # generate equatorial grid, 9 positions
# array_grid_type, array_sh_order = "EMA", 8     # generate equatorial grid, 17 positions
# array_grid_type, array_sh_order = "EMA", 12    # generate equatorial grid, 25 positions
# array_grid_type, array_sh_order = "EMA", 29    # generate equatorial grid, 59 positions
# array_grid_type, array_sh_order = "EMA", 44    # generate equatorial grid, 89 positions
# fmt: on

# Determine spherical sampling grid
if "EMA" in array_grid_type.upper():
    # Generate spherical coordinates as [azimuth, elevation, radius] in rad
    array_coords = np.zeros((3, 2 * array_sh_order + 1))
    array_coords[0] = np.linspace(
        start=0, stop=2 * np.pi, num=array_coords.shape[-1], endpoint=False
    )
    array_coords[2, :] = array_r

    # Generate quadrature weights (sum normalized to 1)
    array_weights = np.ones((array_coords.shape[-1])) / array_coords.shape[-1]

else:
    if "GL" in array_grid_type.upper():
         array_grid = sph_gaussian(
            sh_order=array_sh_order, radius=array_r)
    elif "LE" in array_grid_type.upper():
        array_grid = sph_lebedev(
            sh_order=array_sh_order, radius=array_r)
    elif "FM" in array_grid_type.upper():
        array_grid = sph_fliege(
            sh_order=array_sh_order, radius=array_r)
    else:
        sys.exit(f'Unknown spherical sampling grid type "{array_grid_type}".')

    # Gather spherical coordinates
    array_coords = array_grid.get_sph(
        convention="top_elev", unit="rad"
    ).T  # as [azimuth, elevation, radius] in rad

    # Gather quadrature weights
    array_weights = array_grid.weights
    del array_grid

# Determine coordinate sort indices by ascending azimuth and elevation
# (this yields an okay path with respect to elevation movement)
array_coords_sort_ids = np.lexsort((array_coords[1], array_coords[0]))

# # Alternatively, determine coordinate sort indices by ascending azimuth
# # (this yields an inefficient path with respect to elevation movement)
# array_coords_sort_ids = np.argsort(array_coords[0])

# Plot grid raw and sorted
array_name = f"{array_grid_type.upper()}{array_coords.shape[-1]}"
utils.plot_grid(coords_az_el_r_rad=array_coords, title=f"{array_name} raw")
utils.plot_grid(
    coords_az_el_r_rad=array_coords[:, array_coords_sort_ids],
    title=f"{array_name} sorted",
    is_show_lines=True,
)

# Transform azimuth and elevation coordinates into deg
array_coords[:2] = np.rad2deg(array_coords[:2])
del array_r

## Perform measurement series

Adjust your settings to the individual measurement conditions!

In [ ]:
%matplotlib inline

# Measurement configuration
out_file_name = f"data/{array_name}/{array_name}" + \
    "_pos_{:04d}_az_{:03.0f}_el_{:03.0f}.mat"
meas_delay_duration = 1.0  # in s, duration of initial delay before starting to measure
meas_show_preview = False  # if preview of recorded signals and impulse responses is shown

# Enter manually or add later, e.g. from an `Air CO2ntrol 5000` unit!
air_temperature = None  # in degrees Celsius
air_humidity = None  # in percent, relative humidity

# Estimate based on the used measurement grid/parameters (okay to keep default)!
vari_duration = [1.1, 1.0]  # in s, duration of [move, post-halt-time]
post_duration = [2.0, 1.0]  # in s, duration of post-processing [per iteration, per position]

# Create output data path if it does not exist
Path(out_file_name).parent.mkdir(parents=True, exist_ok=True)

# Check that the recording latency has been determined
try:
    data_rec_latency
except NameError:
    sys.exit(
        'Recording latency is undefined. Execute the "Test audio hardware" section to determine it!'
    )

# Print time estimation
total_iter = array_coords.shape[-1] * meas_iterations
total_meas = meas_delay_duration + np.ceil(total_iter * (
    np.sum(vari_duration)
    + (data_out.shape[0] / fs)
    + post_duration[0]
    + (post_duration[1] / meas_iterations)
))
display(Markdown("<h2><center>Estimated measurement duration is {}</center></h2>".format(
    utils.generate_time_estimation_string(timedelta(seconds=total_meas)))))
time_start = datetime.now()

if meas_show_preview:
    display(widget_preview_rec)
    widget_preview_rec.clear_output()
    display(widget_preview_ir)
    widget_preview_ir.clear_output()

# Wait before starting the measurement
sleep(meas_delay_duration)  # in s

try:
    for cur_id in range(len(array_coords_sort_ids)):
        grid_id = array_coords_sort_ids[cur_id]
        az, el = array_coords[0, grid_id], array_coords[1, grid_id]
        # print(f"{cur_id=}, {grid_id=}, {az=}, {el=}")

        print(f"\nMove VariSphear to {az=:.1f}°, {el=:.1f}° ... ", end="")
        vari.move_blocking(az=az, el=el)
        print("done.")

        # Wait for vibrations to settle
        sleep(vari_duration[1])  # in s

        # Prepare data arrays
        data_rec = np.zeros(
            (
                meas_iterations,
                len(device_input_ch),
                data_out.shape[-1] - data_rec_latency,
            )
        )
        data_ir = np.zeros((meas_iterations, len(
            device_input_ch) - 1, deconv_ir_len))
        data_time = []

        for i in range(meas_iterations):
            pos_str = f"{az=:.1f}°, {el=:.1f}° [{meas_iterations * cur_id + i + 1}/{total_iter}]"

            # Remember measurement time (similar, but not identical to `.isoformat()`)
            data_time.append(datetime.now().strftime("%Y-%m-%dT%H%M%S"))

            while True:
                try:
                    print(f"Record sweep for {pos_str} ... ", end="")
                    with utils.timeout(4 * data_out.shape[0] / fs):
                        rec = sd.playrec(                     # as [iterations, channels, samples]
                            data=data_out,                    # as [samples,]
                            samplerate=fs,                    # in Hz
                            device=device_name,
                            input_mapping=device_input_ch,    # channel numbers start from 1
                            output_mapping=device_output_ch,  # channel numbers start from 1
                            blocking=True,
                        ).T                                   # as [channels, samples]
                    print("done.")
                    continue
                except TimeoutError as e:
                    print(f"{e} ... will try again.")

            # Cut initial zeroes based on determined latency
            data_rec[i, :] = rec[:, data_rec_latency:]  # in samples

            # Print recorded peak and individual RMS levels
            lvl_str = utils.generate_levels_string(data_rec[i])
            print(lvl_str, flush=True)

            if not np.count_nonzero(data_rec[i, :1]):
                print('Loopback signal is all zeros (deconvolution aborted).',
                      file=sys.stderr, flush=True)
                continue
            if not np.count_nonzero(data_rec[i, 1:]):
                print('Microphone signal is all zeros (deconvolution aborted).',
                      file=sys.stderr, flush=True)
                continue

            data_ir[i] = deconvolve(             # as [iterations, channels-1, samples]
                exc_sig=data_rec[i, :1],         # first channel as loopback
                rec_sig=data_rec[i, 1:],
                fs=fs,                           # in Hz
                f_low=deconv_hp,                 # in Hz
                f_high=deconv_lp,                # in Hz
                res_len=deconv_ir_len / fs,      # in s
                deconv_type=deconv_type,
                inv_dyn_db=deconv_inv_dyn,       # in dB
                win_in_len=deconv_win_in_len,    # in samples
                win_out_len=deconv_win_out_len,  # in samples
            )                                    # as [channels, samples]

            if meas_show_preview:
                # Plot data preview (updated after each measurement)
                with widget_preview_rec:
                    widget_preview_rec.clear_output(wait=True)
                    utils.plot_data(data=data_rec[i], fs=fs,
                                    ch_labels=device_input_ch)
                    display(Markdown(f"<h4><center>{lvl_str}</center></h4>"))
                    display(Markdown(widget_preview_str.format(
                        f"Preview individual signals for {pos_str}")))
                    # plot_data(data=data_ir[i], fs=fs,
                    #           ch_labels=device_input_ch[1:])
                    # display(Markdown(widget_preview_str.format(
                    #     f"Preview individual IRs for {pos_str}")))

        # Average deconvolved IRs
        data_ir = np.mean(data_ir, axis=0)  # as [channels-1, samples]

        # Apply desired amplification of impulse responses
        data_ir *= 10 ** (deconv_ir_amp / 20)

        if meas_show_preview:
            # Plot averaged IRs preview (updated after each number of iterations)
            with widget_preview_ir:
                widget_preview_ir.clear_output(wait=True)
                try:
                    utils.plot_data(
                        data=data_ir,
                        fs=fs,
                        ch_labels=device_input_ch[1:],
                        is_stacked=data_ir.ndim > 1 and data_ir.shape[0] > 1,
                    )
                    # noinspection PyUnboundLocalVariable
                    display(Markdown(widget_preview_str.format(
                        f"Preview averaged IRs for {pos_str}")))
                except SystemExit as e:
                    display(Markdown(widget_preview_str.format(e)))

        # Store data in smaller data format (single precision or integer) to lower storage space
        file_name = out_file_name.format(grid_id, az, el)
        print(f'Save data to "{file_name}" ... ', end="")
        savemat(
            file_name,
            {  # fmt: off
                "signal_struct": "[samples, channels, iterations]",
                "signal": data_rec.T.astype(np.float32),            # as [samples, channels, iterations]
                "ir": data_ir.T.astype(np.float32),                 # as [samples, channels-1]
                "fs": np.float32(fs),                               # in Hz
                "air_temperature": np.float32(air_temperature),     # in degrees Celsius
                "air_humidity": np.float32(air_humidity),           # in percent
                "ch_input": np.int16(device_input_ch),              # as list
                "ch_output": np.int16(device_output_ch),            # as list
                "dev_input": device_input_desc,
                "dev_output": device_output_desc,
                "dev_interface": device_name,
                "sh_order": np.int16(array_sh_order),               # as spherical harmonics order
                "r": np.float32(array_coords[2, grid_id]),          # in m
                "scatter_r": np.float32(array_coords[2, grid_id]),  # in m
                "az_deg": np.float32(az),                           # in deg
                "el_deg": np.float32(el),                           # in deg
                "grid_weight": np.float32(array_weights[grid_id]),  # as quadrature weight
                "grid_id": np.int16(grid_id),                       # as original position index
                "meas_id": np.int16(cur_id),                        # as measurement position index
                "meas_room": meas_room,
                "meas_comment": meas_comment,
                "meas_time": data_time,
                "deconv_config": {  # respective `AKdeconv()` configuration
                    "deconv_type": deconv_type,
                    "lowPass": np.float32(deconv_lp),               # in Hz
                    "highPass": np.float32(deconv_hp),              # in Hz
                    "x_inv_dyn": np.float32(deconv_inv_dyn),        # in dB
                    "h_trunc": np.int16(deconv_ir_len),             # in samples
                    "h_fade_in": np.int16(deconv_win_in_len),       # in samples
                    "h_fade_out": np.int16(deconv_win_out_len),     # in samples
                },  # fmt: on
            },
            do_compression=True,
        )
        print("done.")

    time_end = timedelta(seconds=np.ceil(
        (datetime.now() - time_start).seconds))
    display(Markdown("<h2><center>Measurement completed after {}</center></h2>".format(
        utils.generate_time_estimation_string(time_end, is_prediction=False))))

    print(f"\nReset VariSphear ... ", end="")
    vari.move_blocking(az=0, el=0)
    print("done.")

except KeyboardInterrupt:
    print("Interrupted by user.", file=sys.stderr)
